# 使用块的网络（VGG）
---

VGG块

In [1]:
import torch
from torch import nn
from d2l import torch as d2l 

def vgg_block(num_convs, in_channels, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(
            in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers)
        

VGG网络

In [2]:
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512)) #(卷积层数，输出通道数)

def vgg(conv_arch):
    conv_blks = []
    in_channels = 1
    for (num_convs, out_channels) in conv_arch:
        conv_blks.append(vgg_block(
            num_convs, in_channels, out_channels))
        in_channels = out_channels
    return nn.Sequential(
        *conv_blks, nn.Flatten(),
        nn.Linear(out_channels * 7 * 7, 4096), nn.ReLU(),
        nn.Dropout(0.5), nn.Linear(4096, 4096), nn.ReLU(),
        nn.Dropout(0.5), nn.Linear(4096, 10)
    )
net = vgg(conv_arch)

观察每个层输出的形状

In [3]:
X = torch.randn(size=(1, 1, 224, 224))
for blk in net:
    X = blk(X)
    print(blk.__class__.__name__, 'out_put shape:\t', X.shape)

Sequential out_put shape:	 torch.Size([1, 64, 112, 112])
Sequential out_put shape:	 torch.Size([1, 128, 56, 56])
Sequential out_put shape:	 torch.Size([1, 256, 28, 28])
Sequential out_put shape:	 torch.Size([1, 512, 14, 14])
Sequential out_put shape:	 torch.Size([1, 512, 7, 7])
Flatten out_put shape:	 torch.Size([1, 25088])
Linear out_put shape:	 torch.Size([1, 4096])
ReLU out_put shape:	 torch.Size([1, 4096])
Dropout out_put shape:	 torch.Size([1, 4096])
Linear out_put shape:	 torch.Size([1, 4096])
ReLU out_put shape:	 torch.Size([1, 4096])
Dropout out_put shape:	 torch.Size([1, 4096])
Linear out_put shape:	 torch.Size([1, 10])


由于VGG-11比AlexNet计算量更大，因此我们构造了一个通道数较少的网络

In [4]:
ratio = 4
small_conv_arch = [(pair[0], pair[1] // ratio) for pair in conv_arch]
net = vgg(small_conv_arch)

In [ ]:
lr, num_epochs, batch_size = 0.05, 10, 128
train_it